# Using Azure Open Datasets in Synapse - Enrich NYC Green Taxi Data with Holiday and Weather

Synapse has [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/) package pre-installed. This notebook provides examples of how to enrich NYC Green Taxi Data with Holiday and Weather with focusing on :
- read Azure Open Dataset
- manipulate the data to prepare for further analysis, including column projection, filtering, grouping and joins etc. 
- create a Spark table to be used in other notebooks for modeling training

## Data loading 
Let's first load the [NYC green taxi trip records](https://azure.microsoft.com/en-us/services/open-datasets/catalog/nyc-taxi-limousine-commission-green-taxi-trip-records/). The Open Datasets package contains a class representing each data source (NycTlcGreen for example) to easily filter date parameters before downloading.

In [1]:
// Load nyc green taxi trip records from azure open dataset
val blob_account_name = "azureopendatastorage"

val nyc_blob_container_name = "nyctlc"
val nyc_blob_relative_path = "green"
val nyc_blob_sas_token = ""

val nyc_wasbs_path = f"wasbs://$nyc_blob_container_name@$blob_account_name.blob.core.windows.net/$nyc_blob_relative_path"
spark.conf.set(f"fs.azure.sas.$nyc_blob_container_name.$blob_account_name.blob.core.windows.net",nyc_blob_sas_token)

val nyc_tlc = spark.read.parquet(nyc_wasbs_path)

//nyc_tlc.show(5, truncate = false)

In [2]:
// Filter data by time range
import java.sql.Timestamp
import org.joda.time.DateTime

val end_date = new Timestamp(DateTime.parse("2018-06-06").getMillis)
val start_date = new Timestamp(DateTime.parse("2018-05-01").getMillis)

val nyc_tlc_df = nyc_tlc.filter((nyc_tlc("lpepPickupDatetime") >= start_date) && (nyc_tlc("lpepPickupDatetime") <= end_date)) 
nyc_tlc_df.show(5, truncate = false)

Now that the initial data is loaded. Let's do some projection on the data to 
- create new columns for the month number, day of month, day of week, and hour of day. These info is going to be used in the training model to factor in time-based seasonality.
- add a static feature for the country code to join holiday data. 

In [3]:
// Extract month, day of month, and day of week from pickup datetime and add a static column for the country code to join holiday data. 
import org.apache.spark.sql.functions._

val nyc_tlc_df_expand = (
                        nyc_tlc_df.withColumn("datetime", to_date(col("lpepPickupDatetime")))
                                  .withColumn("month_num",month(col("lpepPickupDatetime")))
                                  .withColumn("day_of_month",dayofmonth(col("lpepPickupDatetime")))
                                  .withColumn("day_of_week",dayofweek(col("lpepPickupDatetime")))
                                  .withColumn("hour_of_day",hour(col("lpepPickupDatetime")))
                                  .withColumn("country_code",lit("US"))
                        )

In [4]:
// Display 5 rows
// nyc_tlc_df_expand.show(5, truncate = false)

Remove some of the columns that won't need for modeling or additional feature building.




In [5]:
// Remove unused columns from nyc green taxi data
val nyc_tlc_df_clean = nyc_tlc_df_expand.drop(
                    "lpepDropoffDatetime", "puLocationId", "doLocationId", "pickupLongitude", 
                     "pickupLatitude", "dropoffLongitude","dropoffLatitude" ,"rateCodeID", 
                     "storeAndFwdFlag","paymentType", "fareAmount", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType" )

In [6]:
// Display 5 rows
nyc_tlc_df_clean.show(5, truncate = false)

## Enrich with holiday data
Now that we have taxi data downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. 

Let's load the [public holidays](https://azure.microsoft.com/en-us/services/open-datasets/catalog/public-holidays/) from Azure Open datasets.


In [7]:
// Load public holidays data from azure open dataset
val hol_blob_container_name = "holidaydatacontainer"
val hol_blob_relative_path = "Processed"
val hol_blob_sas_token = ""

val hol_wasbs_path = f"wasbs://$hol_blob_container_name@$blob_account_name.blob.core.windows.net/$hol_blob_relative_path"
spark.conf.set(f"fs.azure.sas.$hol_blob_container_name.$blob_account_name.blob.core.windows.net",hol_blob_sas_token)

val hol_raw = spark.read.parquet(hol_wasbs_path)

// Filter data by time range
val hol_df = hol_raw.filter((hol_raw("date") >= start_date) && (hol_raw("date") <= end_date))

// Display 5 rows
// hol_df.show(5, truncate = false)

Rename the countryRegionCode and date columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. 

In [8]:
val hol_df_clean = (
                hol_df.withColumnRenamed("countryRegionCode","country_code")
                .withColumn("datetime",to_date(col("date")))
                )

hol_df_clean.show(5, truncate = false)

Next, join the holiday data with the taxi data by performing a left-join. This will preserve all records from taxi data, but add in holiday data where it exists for the corresponding datetime and country_code, which in this case is always "US". Preview the data to verify that they were merged correctly.

In [9]:
// enrich taxi data with holiday data
val nyc_taxi_holiday_df = nyc_tlc_df_clean.join(hol_df_clean, Seq("datetime", "country_code") , "left")

nyc_taxi_holiday_df.show(5,truncate = false)

In [10]:
// Create a temp table and filter out non empty holiday rows

nyc_taxi_holiday_df.createOrReplaceTempView("nyc_taxi_holiday_df")
val result = spark.sql("SELECT * from nyc_taxi_holiday_df WHERE holidayName is NOT NULL ")
result.show(5, truncate = false)

## Enrich with weather data¶

Now we append NOAA surface weather data to the taxi and holiday data. Use a similar approach to fetch the [NOAA weather history data](https://azure.microsoft.com/en-us/services/open-datasets/catalog/noaa-integrated-surface-data/) from Azure Open Datasets. 

In [11]:
// Load weather data from azure open dataset
val weather_blob_container_name = "isdweatherdatacontainer"
val weather_blob_relative_path = "ISDWeather/"
val weather_blob_sas_token = ""

val weather_wasbs_path = f"wasbs://$weather_blob_container_name@$blob_account_name.blob.core.windows.net/$weather_blob_relative_path"
spark.conf.set(f"fs.azure.sas.$weather_blob_container_name.$blob_account_name.blob.core.windows.net",hol_blob_sas_token)

val isd = spark.read.parquet(weather_wasbs_path)

// Display 5 rows
// isd.show(5, truncate = false)

In [12]:
// Filter data by time range
val isd_df = isd.filter((isd("datetime") >= start_date) && (isd("datetime") <= end_date))

// Display 5 rows
isd_df.show(5, truncate = false)

In [ ]:
// Filter out weather info for new york city, remove the recording with null temperature 

val weather_df = (
                isd_df.filter(isd_df("latitude") >= "40.53")
                        .filter(isd_df("latitude") <= "40.88")
                        .filter(isd_df("longitude") >= "-74.09")
                        .filter(isd_df("longitude") <= "-73.72")
                        .filter(isd_df("temperature").isNotNull)
                        .withColumnRenamed("datetime","datetime_full")
                        )

In [ ]:
// Remove unused columns
val weather_df_clean = weather_df.drop("usaf", "wban", "longitude", "latitude").withColumn("datetime", to_date(col("datetime_full")))

//weather_df_clean.show(5, truncate = false)

Next group the weather data so that you have daily aggregated weather values. 


In [ ]:
// Enrich weather data with aggregation statistics

val weather_df_grouped = (
                        weather_df_clean.groupBy('datetime).
                        agg(
                            mean('snowDepth) as "avg_snowDepth",
                            max('precipTime) as "max_precipTime",
                            mean('temperature) as "avg_temperature",
                            max('precipDepth) as "max_precipDepth"
                            )
                        )

weather_df_grouped.show(5, truncate = false)

Merge the taxi and holiday data you prepared with the new weather data. This time you only need the datetime key, and again perform a left-join of the data. Run the describe() function on the new dataframe to see summary statistics for each field.

In [ ]:
// Enrich taxi data with weather
val nyc_taxi_holiday_weather_df = nyc_taxi_holiday_df.join(weather_df_grouped, Seq("datetime") ,"left")
nyc_taxi_holiday_weather_df.cache()

In [ ]:
nyc_taxi_holiday_weather_df.show(5,truncate = false)

In [ ]:
// Run the describe() function on the new dataframe to see summary statistics for each field.
display(nyc_taxi_holiday_weather_df.describe())

The summary statistics shows that the totalAmount field has negative values, which don't make sense in the context.



In [ ]:
// Remove invalid rows with less than 0 taxi fare or tip
val final_df = (
            nyc_taxi_holiday_weather_df.
            filter(nyc_taxi_holiday_weather_df("tipAmount") > 0).
            filter(nyc_taxi_holiday_weather_df("totalAmount") > 0)
            )

## Cleaning up the existing Database

First we need to drop the tables since Spark requires that a database is empty before we can drop the Database.

Then we recreate the database and set the default database context to it.

In [ ]:
spark.sql("DROP TABLE IF EXISTS NYCTaxi.nyc_taxi_holiday_weather"); 

In [ ]:
spark.sql("DROP DATABASE IF EXISTS NYCTaxi"); 
spark.sql("CREATE DATABASE NYCTaxi"); 
spark.sql("USE NYCTaxi");

## Creating a new table
We create a nyc_taxi_holiday_weather table from the nyc_taxi_holiday_weather dataframe.


In [ ]:
final_df.write.saveAsTable("nyc_taxi_holiday_weather");
val final_results = spark.sql("SELECT COUNT(*) FROM nyc_taxi_holiday_weather");
final_results.show(5, truncate = false)